In [10]:
class SDES_cipher:
    def __init__(self, Key, IP, IPI, P10, P8, EXP, P4, S0, S1):
#         self.plainText = plainText
        self.Key = Key
        self.IP = IP
        self.IPI = IPI
        self.P10 = P10
        self.P8 = P8
        self.EXP = EXP
        self.P4 = P4
        self.S0 = S0 
        self.S1 = S1
        
    def keyGen(self):
        K = []
        for idx in P10:
            K.append(Key[idx-1])
        shiftK1 = []
        shiftK2 = []
        for i in range(0,10):
            if i < 5:
                shiftK1.append(K[(i+1)%5])
            else:
                shiftK1.append(K[((i+1)%5)+5])
            if i < 5:
                shiftK2.append(K[(i+3)%5])
            else:
                shiftK2.append(K[((i+3)%5)+5])
        K1 = []
        K2 = []
        for idx in P8:
            K1.append(shiftK1[idx-1])
            K2.append(shiftK2[idx-1])
        return K1,K2
    
    def permText(self, plainText):
        PP = []
        for idx in IP:
            PP.append(plainText[idx-1])
        return PP
    
    def expand(self,right):
        RF = []
        for idx in EXP:   
            RF.append(right[idx-1])
        return RF

    def exor(self,right,K):
        for i in range(0,8):
            right[i] = int(right[i])^K[i]
        return right
    
    def funcF(self,l,r):
        L = []
        for i in range (0,4):
            L.append(int(l[i])^r[i])
        return L
    
    def Sboxes(self,bitss):
        idx = bitss[0]*2+bitss[3]*1
        jdx = bitss[1]*2+bitss[2]*1
        SB1 = S0[idx][jdx]
        idx = bitss[4]*2+bitss[7]*1
        jdx = bitss[5]*2+bitss[6]*1
        SB2 = S1[idx][jdx]
        SBL = []
        for item in SB1:
            SBL.append(item)
        for item in SB2:
            SBL.append(item)
        SBLL = []
        for idx in P4:
            SBLL.append(SBL[idx-1])
        return SBLL

In [11]:
text = 'sumitpatil'
plainText = []
for item in text:
    value = (ord(item)-97)%26
    plainText.append("{0:008b}".format(value))
    
sizeT = len(text)

In [12]:
Key = [1,1,0,0,0,1,1,1,1,0]
IP = [2,6,3,1,4,8,5,7]
IPI = [4,1,3,5,7,2,8,6]
P10 = [3,5,2,7,4,10,1,9,8,6]
P8 = [6,3,7,4,8,5,10,9]
EXP = [4,1,2,3,2,3,4,1]
P4 = [2,4,3,1]
S0 = [[(0,1),(0,0),(1,1),(1,0)],
      [(1,1),(1,0),(0,1),(0,0)],
      [(0,0),(1,0),(0,1),(1,1)],
      [(1,1),(0,1),(1,1),(1,0)]]
S1 = [[(0,0),(0,1),(1,0),(1,1)],
      [(1,0),(0,0),(0,1),(1,1)],
      [(1,1),(0,0),(0,1),(0,0)],
      [(1,0),(0,1),(0,0),(1,1)]]

In [13]:
cipher2 = SDES_cipher(Key,IP,IPI,P10,P8,EXP,P4,S0,S1)
K1, K2 = cipher2.keyGen()
print('Key 1 :', K1)
print('Key 2 :', K2)
print()

Key 1 : [1, 1, 1, 0, 1, 0, 0, 1]
Key 2 : [1, 0, 1, 0, 0, 1, 1, 1]



In [14]:
cipherText = []
print('Message to be coded :', text)
print('Message in binary format :',plainText)
print()
i = 1
for item in plainText:
    print('--------BLOCK',i,'----------')
    i = i + 1
    text = []
    text[:0] = item
    permutedText = cipher2.permText(text)
    print('Permuted Text is :', permutedText)

    # Round 1
    print()
    print('----------ROUND 1---------')
    L,R = permutedText[:4], permutedText[4:]
    print('Left :', L)
    print('Right :', R)

    RE1 = cipher2.expand(R)
    print('Expanded Right :', RE1)

    RE1 = cipher2.exor(RE1,K1)
    print('Right^Key :', RE1)

    RE11 = cipher2.Sboxes(RE1)
    print('After applying Sboxes and permuting result:', RE11)

    LE11 = cipher2.funcF(L,RE11)
    print('F() :', LE11)

    # Round2
    print()
    print('----------ROUND 2---------')
    L2,R2= R,LE11
    print('Left :', L2)
    print('Right :', R2)

    RE2 = cipher2.expand(R2)
    print('Expanded Right :', RE2)

    RE2 = cipher2.exor(RE2,K2)
    print('Right^Key :', RE2)

    RE22 = cipher2.Sboxes(RE2)
    print('After applying Sboxes and permuting result:', RE22)

    LE22 = cipher2.funcF(L2,RE22)
    print('F() :', LE22)

    #Final merging
    final = []
    for item in LE22:
        final.append(item)
    for item in R2:
        final.append(item)

    for idx in IPI:
        cipherText.append(final[idx-1])
    print()
    
print('Cipher Text obtained is :',cipherText)
print()

Message to be coded : sumitpatil
Message in binary format : ['00010010', '00010100', '00001100', '00001000', '00010011', '00001111', '00000000', '00010011', '00001000', '00001011']

--------BLOCK 1 ----------
Permuted Text is : ['0', '0', '0', '0', '1', '0', '0', '1']

----------ROUND 1---------
Left : ['0', '0', '0', '0']
Right : ['1', '0', '0', '1']
Expanded Right : ['1', '1', '0', '0', '0', '0', '1', '1']
Right^Key : [0, 0, 1, 0, 1, 0, 1, 0]
After applying Sboxes and permuting result: [0, 0, 0, 0]
F() : [0, 0, 0, 0]

----------ROUND 2---------
Left : ['1', '0', '0', '1']
Right : [0, 0, 0, 0]
Expanded Right : [0, 0, 0, 0, 0, 0, 0, 0]
Right^Key : [1, 0, 1, 0, 0, 1, 1, 1]
After applying Sboxes and permuting result: [0, 1, 1, 1]
F() : [1, 1, 1, 0]

--------BLOCK 2 ----------
Permuted Text is : ['0', '1', '0', '0', '1', '0', '0', '0']

----------ROUND 1---------
Left : ['0', '1', '0', '0']
Right : ['1', '0', '0', '0']
Expanded Right : ['0', '1', '0', '0', '0', '0', '0', '1']
Right^Key : 

In [15]:
decipher2 = SDES_cipher(Key,IP,IPI,P10,P8,EXP,P4,S0,S1)
K2, K1 = decipher2.keyGen()
print('Key 1 :',K2)
print('Key 2 :',K1)

Key 1 : [1, 1, 1, 0, 1, 0, 0, 1]
Key 2 : [1, 0, 1, 0, 0, 1, 1, 1]


In [16]:
# print(cipherText)
decipherText = []
i = 0
j = 1
while i<(sizeT*8):
    print()
    print('--------BLOCK',j,'----------')
    j = j + 1
    text = cipherText[i:(i+8)]
    permutedText = cipher2.permText(text)
    print('Permuted Text is :', permutedText)

    # Round 1
    print()
    print('----------ROUND 1---------')
    L,R = permutedText[:4], permutedText[4:]
    print('Left :', L)
    print('Right :', R)

    RE1 = cipher2.expand(R)
    print('Expanded Right :', RE1)

    RE1 = cipher2.exor(RE1,K1)
    print('Right^Key :', RE1)

    RE11 = cipher2.Sboxes(RE1)
    print('After applying Sboxes and permuting result:', RE11)

    LE11 = cipher2.funcF(L,RE11)
    print('F() :', LE11)

    # Round2
    print()
    print('----------ROUND 2---------')
    L2,R2= R,LE11
    print('Left :', L2)
    print('Right :', R2)

    RE2 = cipher2.expand(R2)
    print('Expanded Right :', RE2)

    RE2 = cipher2.exor(RE2,K2)
    print('Right^Key :', RE2)

    RE22 = cipher2.Sboxes(RE2)
    print('After applying Sboxes and permuting result:', RE22)

    LE22 = cipher2.funcF(L2,RE22)
    print('F() :', LE22)

    #Final merging
    final = []
    for item in LE22:
        final.append(item)
    for item in R2:
        final.append(item)

    for idx in IPI:
        decipherText.append(final[idx-1])
    print()
    i = i + 8
    
print('Deipher Text obtained is :',decipherText)


--------BLOCK 1 ----------
Permuted Text is : [1, 1, 1, 0, 0, 0, 0, 0]

----------ROUND 1---------
Left : [1, 1, 1, 0]
Right : [0, 0, 0, 0]
Expanded Right : [0, 0, 0, 0, 0, 0, 0, 0]
Right^Key : [1, 0, 1, 0, 0, 1, 1, 1]
After applying Sboxes and permuting result: [0, 1, 1, 1]
F() : [1, 0, 0, 1]

----------ROUND 2---------
Left : [0, 0, 0, 0]
Right : [1, 0, 0, 1]
Expanded Right : [1, 1, 0, 0, 0, 0, 1, 1]
Right^Key : [0, 0, 1, 0, 1, 0, 1, 0]
After applying Sboxes and permuting result: [0, 0, 0, 0]
F() : [0, 0, 0, 0]


--------BLOCK 2 ----------
Permuted Text is : [1, 0, 1, 1, 0, 0, 1, 1]

----------ROUND 1---------
Left : [1, 0, 1, 1]
Right : [0, 0, 1, 1]
Expanded Right : [1, 0, 0, 1, 0, 1, 1, 0]
Right^Key : [0, 0, 1, 1, 0, 0, 0, 1]
After applying Sboxes and permuting result: [0, 0, 1, 1]
F() : [1, 0, 0, 0]

----------ROUND 2---------
Left : [0, 0, 1, 1]
Right : [1, 0, 0, 0]
Expanded Right : [0, 1, 0, 0, 0, 0, 0, 1]
Right^Key : [1, 0, 1, 0, 1, 0, 0, 0]
After applying Sboxes and permuting

In [17]:
sblank = " "
i = 0
finalText = []
while i < (sizeT*8):
    text = []
    text = decipherText[i:(i+8)]
    result = int("".join(str(i) for i in text),2)
    result = result + 97
    finalText.append(chr(result))
    i = i + 8

In [18]:
"".join(finalText)

'sumitpatil'